<a href="https://colab.research.google.com/github/phanicode/ucast/blob/main/ucast_stencil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from collections import namedtuple

Site = namedtuple('Site', ['name', 'lat', 'lon', 'alt'])

ALMA = Site('ALMA', -23.029,  -67.755, 5070.4)
APEX = Site('APEX', -23.006,  -67.759, 5104.5)
GLT  = Site('GLT',   76.5312, -68.7039,  76.5)
JCMT = Site('JCMT',  19.823, -155.477, 4120.1)
KP   = Site('KP',    31.956, -111.612, 1902.0)
LMT  = Site('LMT',   18.985,  -97.315, 4600.0)
PDB  = Site('PDB',   44.634,    5.907, 2616.4)
PV   = Site('PV',    37.066,   -3.393, 2921.7)
SMA  = Site('SMA',   19.822, -155.476, 4110.3)
SMT  = Site('SMT',   32.702, -109.891, 3158.7)
SPT  = Site('SPT',  -90.000,   45.000, 2857.4)

In [5]:
import requests
import urllib
import pandas as pd

# USGS Elevation Point Query Service
url = r'https://nationalmap.gov/epqs/pqs.php?'
def elevation_function(lat,lon):
    """Query service using lat, lon. add the elevation values as a new column."""
    lat=[lat]
    lon=[lon]
    for lat, lon in zip(lat,lon):
        # define rest query params
        params = {
            'output': 'json',
            'x': lon,
            'y': lat,
            'units': 'Meters'
        }
        # format query string and return query value
        result = requests.get((url + urllib.parse.urlencode(params)))
        return float(result.json()['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation'])




In [31]:
def get_sites(sites,stencil_size=1,grid_size=0.25):
        for site in sites:
            globals()[f"{site.name}_{stencil_size}_E"] = Site(f"{site.name}_{stencil_size}_E", site.lat,site.lon+0.25*stencil_size,elevation_function(site.lat,site.lon+0.25*stencil_size))
            globals()[f"{site.name}_{stencil_size}_W"] = Site(f"{site.name}_{stencil_size}_W", site.lat,site.lon-0.25*stencil_size,elevation_function(site.lat,site.lon-0.25*stencil_size))
            globals()[f"{site.name}_{stencil_size}_S"] = Site(f"{site.name}_{stencil_size}_S", site.lat-0.25*stencil_size,site.lon,elevation_function(site.lat-0.25*stencil_size,site.lon))
            globals()[f"{site.name}_{stencil_size}_N"] = Site(f"{site.name}_{stencil_size}_N",site.lat+0.25*stencil_size,site.lon,elevation_function(site.lat+0.25*stencil_size,site.lon))
            globals()[f"{site.name}_{stencil_size}_SW"] = Site(f"{site.name}_{stencil_size}_SW",site.lat-0.25*stencil_size,site.lon-0.25*stencil_size,elevation_function(site.lat-0.25*stencil_size,site.lon+0.25*stencil_size))
            globals()[f"{site.name}_{stencil_size}_SE"] = Site(f"{site.name}_{stencil_size}_SE",site.lat-0.25*stencil_size,site.lon+0.25*stencil_size,elevation_function(site.lat+0.25*stencil_size,site.lon+0.25*stencil_size))
            globals()[f"{site.name}_{stencil_size}_NW"] = Site(f"{site.name}_{stencil_size}_NW",site.lat+0.25*stencil_size,site.lon-0.25*stencil_size,elevation_function(site.lat-0.25*stencil_size,site.lon+0.25*stencil_size))
            globals()[f"{site.name}_{stencil_size}_NE"] = Site(f"{site.name}_{stencil_size}_NE",site.lat+0.25*stencil_size,site.lon+0.25*stencil_size,elevation_function(site.lat+0.25*stencil_size,site.lon-0.25*stencil_size))

In [38]:
# Getting a stencil for site. Default value is 1 (for one stencil grid)
get_sites([KP])

In [33]:
import gc
from math import floor
# print(subregion_query(KP.lat,KP.lon))
for obj in gc.get_objects():
    if isinstance(obj, Site) and "KP" in obj.name:
        print(obj)

#Site(name='KP_1_E', lat=31.956, lon=-111.362, alt=901.75)
Site(name='KP_1_W', lat=31.956, lon=-111.862, alt=795.67)
Site(name='KP_1_S', lat=31.706, lon=-111.612, alt=1450.67)
Site(name='KP_1_N', lat=32.206, lon=-111.612, alt=667.91)
Site(name='KP_1_SW', lat=31.706, lon=-111.862, alt=1034.09)
Site(name='KP_1_SE', lat=31.706, lon=-111.362, alt=769.82)
Site(name='KP_1_NW', lat=32.206, lon=-111.862, alt=1034.09)
Site(name='KP_1_NE', lat=32.206, lon=-111.362, alt=671.47)
Site(name='KP', lat=31.956, lon=-111.612, alt=1902.0)


In [ ]:
#North site.lat,site.lon+0.25
#South site.lat,site.lon-0.25
#West site.lat+0.25,site.lon
#East site.lat-0.25,site.lon

#NE site.lat-0.25,site.lon+0.25
#NW site.lat+0.25,site.lon+0.25
#SE site.lat-0.25,site.lon+0.25
#SW site.lat+0.25,site.lon-0.25

In [ ]:
from math import floor

In [9]:
def subregion_query(lat, lon, gridsz=0.25):
    """Query string for the grid subset request."""
    l = floor(lon / gridsz) * gridsz
    b = floor(lat / gridsz) * gridsz
    r = l + gridsz
    t = b + gridsz
    print(l,",",t)
    print(l,",",b)
    print(r,",",t)
    print(r,",",b)
#     print( f"subregion=&leftlon={l}&rightlon={r}&toplat={t}&bottomlat={b}")

original lon -111.362
lon, lon_2: -111.362 -111.5


In [36]:
import gc
from math import floor

# FIT NEW COORDINATES TO GRID
for obj in gc.get_objects():
    if isinstance(obj, Site) and "KP" in obj.name:
        print(obj,subregion_query(obj.lat,obj.lon))

-111.75 , 32.0
-111.75 , 31.75
-111.5 , 32.0
-111.5 , 31.75
Site(name='KP', lat=31.956, lon=-111.612, alt=1902.0) None
-111.5 , 32.0
-111.5 , 31.75
-111.25 , 32.0
-111.25 , 31.75
Site(name='KP_1_E', lat=31.956, lon=-111.362, alt=901.75) None
-112.0 , 32.0
-112.0 , 31.75
-111.75 , 32.0
-111.75 , 31.75
Site(name='KP_1_W', lat=31.956, lon=-111.862, alt=795.67) None
-111.75 , 31.75
-111.75 , 31.5
-111.5 , 31.75
-111.5 , 31.5
Site(name='KP_1_S', lat=31.706, lon=-111.612, alt=1450.67) None
-111.75 , 32.25
-111.75 , 32.0
-111.5 , 32.25
-111.5 , 32.0
Site(name='KP_1_N', lat=32.206, lon=-111.612, alt=667.91) None
-112.0 , 31.75
-112.0 , 31.5
-111.75 , 31.75
-111.75 , 31.5
Site(name='KP_1_SW', lat=31.706, lon=-111.862, alt=1034.09) None
-111.5 , 31.75
-111.5 , 31.5
-111.25 , 31.75
-111.25 , 31.5
Site(name='KP_1_SE', lat=31.706, lon=-111.362, alt=769.82) None
-112.0 , 32.25
-112.0 , 32.0
-111.75 , 32.25
-111.75 , 32.0
Site(name='KP_1_NW', lat=32.206, lon=-111.862, alt=1034.09) None
-111.5 , 32.25